## Практика 1

## Важно! О формате сдачи

🔷 **<font color='plum'>При решении ноутбука используйте данный шаблон</font>**

    ✅ Можно добавлять новые ячейки любых типов
    ❌ Не нужно удалять текстовые ячейки c разметкой частей ноутбука и формулировками заданий


🔷 **<font color='plum'>При оценивании задач учитывается код</font>**

    ✅ Задания, в которых необходим код, обычно помечаются фразами "Your code here"/"Ваш код" и аналогичными
    ❌ Ответы на вопросы без сопутствующего кода оцениваются в 0 баллов
    ❌ Наличе работоспособного кода в ноутбуке, если на сказано иного, обязательно

🔷 **<font color='plum'>При оценивании задач учитываются выводы</font>**

    ✅ Задания, в которых необходимы выводы, обычно помечаются фразами Вывод"/"Ответ на вопрос"/"Ваш текст" и аналогичными
    ✅ Обычно выводы подразумевают под собой текстовый ответ (можно писать markdown, latex).
    ✅ Сопутствующие изображения, графики, таблички - приветствуются!
    ❌ При отсутствии выводов задание не засчитается на полный балл

# Часть 0. Подготовка

<font color='OrangeRed'>**Внимание!**</font> Вам может понадобиться свежая версия <font color='CornflowerBlue'>**sklearn>=1.3**</font>

In [1]:
pip install scikit-learn -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [2]:
import sklearn
print(sklearn.__version__)
assert(sklearn.__version__ >= "1.3.1")

1.8.0


In [3]:
import pandas as pd
import numpy as np

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.io as pio
import matplotlib_inline

import time
from tqdm.auto import tqdm

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.metrics import mean_squared_error, RocCurveDisplay, roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC, SVC


sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
pio.templates.default = pio.templates.merge_templates("plotly_dark", "presentation")
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

pd.set_option('display.max_columns', None)

<font color='DarkSeaGreen'>**Если много warning будет, то можно включить:**</font>

In [4]:
import warnings
warnings.filterwarnings("ignore")

#Линейная регрессия

##  <font color='DarkOrange'>**Задача 1**</font>

<font color='DarkOrange'>**Вопрос (?)**</font> Расскажите, почему при использовании регуляризации штрафуются только веса, но не свободный коэффициент $w_0$ (в некоторой литературе - $b$)?

Напомним, что предсказание линейной модели считается по формуле

$$a(x) = \langle w, x \rangle + w_0$$

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

-------

Данные возьмем из соревнования [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

In [5]:
! gdown 1SKx5Shx5J1O45mKBmjCtQmUF7LXRUou-

Downloading...
From: https://drive.google.com/uc?id=1SKx5Shx5J1O45mKBmjCtQmUF7LXRUou-
To: /content/train.csv
100% 461k/461k [00:00<00:00, 103MB/s]


In [6]:
data = pd.read_csv("/content/train.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


##  <font color='DarkOrange'>**Задача 2**</font>
В домашнем задании нашей основной метрикой будет RMSE. Для начала обучите модель линейной регресии (с помощью модуля `LinearRegression`) только на числовых признаках. Посмотрите, какого значения RMSE удастся достичь на обучающей и тестовой выборке.


In [7]:
X_numeric = data.select_dtypes(include='number').drop(["SalePrice"], axis=1).fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X_numeric, data.SalePrice, test_size=0.1, random_state=42)

# Ваш код тут

##  <font color='DarkOrange'>**Задача 3**</font>
Давайте попробуем отнормировать наши численные признаки и посмотрим, как после этого изменится RMSE на тренировочной и обучающей выборке.
Для этой задачи будем использовать `StandardScaler`.


In [8]:
scaler = StandardScaler()

# Ваш код тут

##  <font color='DarkOrange'>**Задача 4**</font>
Обработайте пропущенные данные таким образом, чтобы в дальнейшем на их основе вы могли обучить модель. После этого разбейте данные на обучающую и тестовую выборку.

In [9]:
# Ваш код тут

##  <font color='DarkOrange'>**Задача 5**</font>
Постройке box-plot по нашему таргету (SalePrice). Какие выводы можно сделать на основе полученной визуализации?

<font color='MediumOrchid'>**Ваш ответ:**</font>

In [10]:
# Ваш код тут

##  <font color='DarkOrange'>**Задача 6**</font>
Давайте избавимся от части выбросов. Для этого постройте новый box-plot, внутри которого будут лежать значения из интервала ниже: $$[ Q_1 -1.5IQR, Q_3 + 1.5IQR ]$$

где $IQR$ - интерквартильный размах, $Q_1$ - первая квантиль, $Q_3$ - третья квантиль.

Что произошло с выбросами и почему?

<font color='MediumOrchid'>**Ваш ответ:**</font>

In [11]:
# Ваш код тут

##  <font color='DarkOrange'>**Задача 7**</font>
Рассчитайте количество уникальных значений для каждого категориального признака.

In [12]:
# Ваш код тут

##  <font color='DarkOrange'>**Задача 8**</font>
Закодируйте категориальные признаки, используя One-Hot encoding (можно использовать любую реализацию).

In [13]:
# Ваш код тут

##  <font color='DarkOrange'>**Задача 9**</font>

<font color='plum'>**Проведите 2 эксперимента**</font>

1) Подберите оптимальные параметры (`alpha` и `l1_ratio`) с помощью `GridSearchCV`, `Pipeline` для модели `ElasticNet`. Для нормализации данных используйте `MinMaxScaler`

2) Подберите оптимальные параметры (`alpha` и `l1_ratio`) с помощью `GridSearchCV`, `Pipeline` для модели `ElasticNet`. Для нормализации данных используйте `StandardScaler`

В качестве метрики качества будем `по-прежнему использовать RMSE`

<font color='plum'>**Ответьте на несколько вопросов**</font>

* Выведите оптимальные параметры в двух экспериментах. Отличаются ли они?
* Какое получилось качество на тесте в этих двух случаях?
* Какие признаки получилось занулить в каждом из экспериментов? Как вы думаете, почему они занулились? (Можно [посмотреть на описание признаков посмотреть вот тут](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=test.csv) и [тут](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=data_description.txt))
* Сделайте вывод об влиянии типа нормализатора на качество решения линейной регрессии



<font color='OrangeRed'>**Внимание!**</font> помните, что порядок наполнения `Pipeline` нужно конструировать с учетом избежания утечек целевой переменной! Подробнее об этом можно посмотреть в семинарском ноутбуке.

<font color='LightSteelBlue'>**Подсказка:**</font> Хотя бы в одном из экспериментов должно занулиться хотя бы 2 признака.  Если этого не произошло, перепроверьте сетку перебора или разбиения на `train_test_split`

<font color='LightSteelBlue'>**Рекомендации:**</font>
* `alpha` рекомендуется перебирать про логарифмической шкале, хотя бы 10 значений
* `l1_ratio` - по линейной шкале, включая границы 0 и 1 (границы значений [можно посмотреть тут](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) ), хотя бы 5 значений

In [14]:
# Ваш код тут

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

##  <font color='DarkOrange'>**Задача 10**</font>

Теперь попробуем сделать следующее: обучить `Lasso` на данных из предыдущей задачи и посмотреть поведение количества зануляемых весов в зависимости от силы регуляризации.

* Изобразите график количества зануляемых весов в зависимости от коэффициента регуляризации `alpha`. Можно пользоваться любыми библиотеками визуализации
* Какой характер имеет зависимость?

In [15]:
# Ваш код тут

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

#  <font color='MediumSeaGreen'>**Бонусное задание**</font>

Пусть имеется метрика качества

$$\text{RMSLE}(X, y, a) = \sqrt{\frac{1}{N}\sum_{i=1}^{N} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2}$$

$N$ - размер датасета

$y_i$ - таргет в i-м объекте

$a(x_i)$ - предсказания алгоритма $a$ в объкте $x_i$. $a$ обучается на таргет $y_i$


Пусть $\widehat{y}_i = \log{(y_i + 1)}$ - модифицированный таргет, $\widehat{a}(x_i)$ - предсказание модели $\widehat{a}$ на объекта $x_i$, модель обучалась на таргет $\widehat y$.

Таким образом $$a(x_i) = \exp(\widehat{a}(x_i)) - 1$$

<font color='DarkOrange'>**Докажите математически:**</font> Оптимизация RMSLE на оригинальном таргете $y$ и модели $a$ эквивалентна оптимизации MSE на таргете $\widehat y$ и модели $\widehat a$

Предпочтительно доказательство набрать в колабе в текстовой ячейке, через LaTeX команды. Можно также приложить картинку с рукописным выводом или ссылку на отдельный pdf-файлик в комментариях при сдаче задания в lms

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

#  <font color='MediumSeaGreen'>**Бонусное задание**</font>

*  Какое ваше любимое блюдо/еда? Пришлите его фоточку или рецепт или ресторан/кафе/и пр., где его готовят